# TCC PUC-Minas Gastão Jucá Filho <br>
Turma Puc-Minas Virtual - nov2019 <br>
Notebook executado no <font color=green>final de semana 16 a 19 de abril de 2021</font>

#  Atividades:
> 1. Continuar o notebook "Tarefas até 16abr2021". 
> 2. "Limpar" as tabelas restantes da base da Olist. As tabelas a limpar são:
>> <font color=red>"customer"</font>, <font color=red>"payment"</font>, <font color=red>"product"</font>, <font color=red>"seller"</font> e <font color=red>"geolocation"</font>;  
> 3. Fazer algum merge. 
> 4. Documentar tudo com linguagem Markdown.

##  1 - Importação de bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.options.display.max_columns = 100
from IPython.display import Image
from IPython.core.display import HTML 

## 2 - Importação das 07 (sete) tabelas 
> (As 3 primeiras já foram trabalhadas).  

> Na verdade, deixei de carregar a tabela <font color=red>"olist_geolocation_dataset.csv"</font> pelo seu tamanho (58 MB) e por ela não ser imprescindível nesta fase.

In [2]:
reviews = pd.read_csv("olist_order_reviews_dataset.csv", sep = ',')

In [3]:
orders = pd.read_csv('olist_orders_dataset.csv', sep=',')

In [4]:
itens = pd.read_csv("olist_order_items_dataset.csv", sep = ',')

In [5]:
products = pd.read_csv("olist_products_dataset.csv", sep = ',')

In [6]:
customers = pd.read_csv("olist_customers_dataset.csv", sep = ',')

In [7]:
# geolocation = pd.read_csv("olist_geolocation_dataset.csv", sep = ',')

In [8]:
payments = pd.read_csv("olist_order_payments_dataset.csv", sep = ',')

In [9]:
sellers = pd.read_csv("olist_sellers_dataset.csv", sep = ',')

## 3 - Resumo do realizado no prévio notebook "Tarefas até 16abr2021"

In [10]:
# Encontramos as tabelas "reviews" e "orders" do jeito abaixo (precisava "limpar" a da esquerda)
Image(url= "Reviews and Orders being looked into.bmp")

In [11]:
# Decidimos deletar os duplicados do campo 'review_id'
review_id_sem_repetir = reviews.drop_duplicates (subset=['review_id'])

In [12]:
# Qtde de registros da tabela passou de 100 mil para 99.173
Image(url= "review_id_sem_repetir e orders.bmp")

In [13]:
# Depois, decidimos deletar os duplicados do campo 'order_id'
reviews_normalizada = review_id_sem_repetir.drop_duplicates (subset=['order_id']) 

In [14]:
# Qtde de registro caiu mais um pouco.
Image(url= "reviews_normalizada e orders.bmp")

#### A tabela <font color=red>"orders"</font> não precisou de limpeza.  
Partimos para a tabela <font color=red>"itens"</font> (à direita na fig. abaixo)

In [15]:
Image (url = "tres tabelas.bmp")

In [16]:
# Optamos por deletar duplicatas do campo "order_item_id"
itens_sem_repetir = itens.drop_duplicates (subset=['order_id']) # Será mantida a 1ª ocorrência.

In [17]:
# situação APÓS da deleção
Image (url = "tres tabelas pos delecao.bmp")

#### Apesar de a nova tabela <font color=red>"itens_sem_repetir"</font> falhar em informar sobre itens, ainda é útil para informar qual o produto e qual o fornecedor estão atrelados a cada pedido.
> No entanto, vamos desprezar em uma futura operação de 'Join' as colunas restantes, pois:  
>> Os preços do produto e do frete influenciariam somente em uma análise de 'Churn' que não é o nosso caso.  
>> A data de envio também é desnecessária, uma vez que temos datas diversas na tabela <font color=red>"orders"</font>.

## 4 -  Agora vamos partir para a limpeza das tabelas restantes.

### 4.1 - Tabela <font color=red>"products"

In [18]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.00,287.00,1.00,225.00,16.00,10.00,14.00
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.00,276.00,1.00,"1,000.00",30.00,18.00,20.00
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.00,250.00,1.00,154.00,18.00,9.00,15.00
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.00,261.00,1.00,371.00,26.00,4.00,26.00
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.00,402.00,4.00,625.00,20.00,17.00,13.00


In [19]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
product_id                    32951 non-null object
product_category_name         32341 non-null object
product_name_lenght           32341 non-null float64
product_description_lenght    32341 non-null float64
product_photos_qty            32341 non-null float64
product_weight_g              32949 non-null float64
product_length_cm             32949 non-null float64
product_height_cm             32949 non-null float64
product_width_cm              32949 non-null float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [20]:
len(products['product_id'].unique())

32951

In [21]:
products['product_id'].nunique()

32951

In [22]:
len(products['product_category_name'].unique()) # unique lista até os NAN

74

In [23]:
products['product_category_name'].nunique() # number of unique não inclui os NAN na contagem

73

#### <font color=magenta>RESUMO</font>. Não há a descrição do nome do produto, somente o seu código. Não podemos ver qual produto é, apenas a sua categoria. Há 74 categorias para os mais de 32 mil produtos.
> 'product_id' é a chave primária da tabela <font color=red>"products"</font>

> Não há o que limpar, a tabela <font color=red>"products"</font> está pronta para uso.

In [24]:
# Nossa quarta tabela se aglutina
Image (url = "tabelas com products.bmp")

### 4.2 - Tabela <font color=red>"payments"

#### 4.2.1 - dados iniciais

In [25]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [26]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
order_id                103886 non-null object
payment_sequential      103886 non-null int64
payment_type            103886 non-null object
payment_installments    103886 non-null int64
payment_value           103886 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [27]:
# Valores únicos dos campos
payments[['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']].nunique()

order_id                99440
payment_sequential         29
payment_type                5
payment_installments       24
payment_value           29077
dtype: int64

In [28]:
# Quais são os 29 'payment_sequential'
df_payments_unique = pd.DataFrame(payments['payment_sequential'].unique(), columns=['payment_sequential'])
df_payments_unique.sort_values(by='payment_sequential')

,payment_sequential
0,1
1,2
4,3
2,4
3,5
6,6
7,7
5,8
14,9
8,10


In [29]:
# Quais são os 5 'payment_type'
df_payment_type = pd.DataFrame(payments['payment_type'].unique(), columns=['payment_type'])
df_payment_type

,payment_type
0,credit_card
1,boleto
2,voucher
3,debit_card
4,not_defined


In [30]:
# Quais são os 24 'payment_installments'
df_payment_installments = pd.DataFrame(payments['payment_installments'].unique(), columns=['payment_installments'])
df_payment_installments.sort_values(by='payment_installments')

,payment_installments
21,0
1,1
2,2
3,3
6,4
5,5
4,6
8,7
0,8
10,9


In [31]:
# Estatística dos valores da compra
payments['payment_value'].describe()

count   103,886.00
mean        154.10
std         217.49
min           0.00
25%          56.79
50%         100.00
75%         171.84
max      13,664.08
Name: payment_value, dtype: float64

In [32]:
Image (url = "cinco tabelas - inclui payments.bmp")

#### 4.2.2 - Investigando o campo 'payment_sequential': qual sua finalidade?  
Resposta:
> Um cliente pode pagar um pedido com mais de um método de pagamento.  

> Se ele fizer isso, uma sequência será criada para acomodar todos os pagamentos. 

In [33]:
# Vamos agrupar 'paymente_sequential' e contar 'order_id'
payment_sequential_agrupados = payments.groupby('payment_sequential')['order_id'].count()

# vamos transformar em um Dataframe o Series resultante desse GroupBy 
df_payment_sequential_agrupados = pd.DataFrame(payment_sequential_agrupados)

# Visualizemos qtos 'order_id' há em cada 'payment_sequential'
df_payment_sequential_agrupados

,order_id
payment_sequential,
1,99360
2,3039
3,581
4,278
5,170
6,118
7,82
8,54
9,43


##### 4.2.2a - Listando alguns 'order_id' com múltiplos modos de pagamento

In [34]:
# Listagem dos 10 (dez)'order_id' que contêm o 'payment_sequential' 14 (obtido acima)
payments.loc[payments['payment_sequential'] == 14]

,order_id,payment_sequential,payment_type,payment_installments,payment_value
9104,21577126c19bf11a0b91592e5844ba78,14,voucher,1,7.11
13960,4689b1816de42507a7d63a4617383c59,14,voucher,1,42.31
16279,ccf804e764ed5650cd8759557269dc13,14,voucher,1,0.97
18175,fedcd9f7ccdc8cba3a18defedd1a5547,14,voucher,1,7.63
19078,285c2e15bebd4ac83635ccc563dc71f4,14,voucher,1,1.75
36822,fa65dad1b0e818e3ccc5cb0e39231352,14,voucher,1,0.00
40166,4bfcba9e084f46c8e3cb49b0fa6e6159,14,voucher,1,50.00
50071,ee9ca989fc93ba09a6eddc250ce01742,14,voucher,1,4.70
86294,895ab968e7bb0d5659d16cd74cd1650c,14,voucher,1,16.70
90473,3c58bffb70dcf45f12bdf66a3c215905,14,voucher,1,3.64


In [35]:
# Listagem de um 'order_id' que contém o 'payment_sequential' 14
payments.loc[payments['order_id'] == "21577126c19bf11a0b91592e5844ba78"].sort_values(by='payment_sequential')

,order_id,payment_sequential,payment_type,payment_installments,payment_value
76153,21577126c19bf11a0b91592e5844ba78,1,voucher,1,3.79
64586,21577126c19bf11a0b91592e5844ba78,2,voucher,1,7.92
67574,21577126c19bf11a0b91592e5844ba78,3,voucher,1,4.18
91141,21577126c19bf11a0b91592e5844ba78,4,voucher,1,5.47
71725,21577126c19bf11a0b91592e5844ba78,5,voucher,1,4.18
96224,21577126c19bf11a0b91592e5844ba78,6,voucher,1,7.11
30664,21577126c19bf11a0b91592e5844ba78,7,voucher,1,4.65
24774,21577126c19bf11a0b91592e5844ba78,8,voucher,1,7.11
40942,21577126c19bf11a0b91592e5844ba78,9,voucher,1,11.33
4439,21577126c19bf11a0b91592e5844ba78,10,voucher,1,9.34


##### Tudo voucher????? Que coisa ridícula e sem sentido.

In [36]:
# Listagem de outro 'order_id' que contém o 'payment_sequential' 14
payments.loc[payments['order_id'] == "4689b1816de42507a7d63a4617383c59"].sort_values(by='payment_sequential')

,order_id,payment_sequential,payment_type,payment_installments,payment_value
54311,4689b1816de42507a7d63a4617383c59,1,credit_card,1,21.81
13997,4689b1816de42507a7d63a4617383c59,2,voucher,1,56.42
1769,4689b1816de42507a7d63a4617383c59,3,voucher,1,56.42
48253,4689b1816de42507a7d63a4617383c59,4,voucher,1,14.10
74588,4689b1816de42507a7d63a4617383c59,5,voucher,1,42.31
87728,4689b1816de42507a7d63a4617383c59,6,voucher,1,42.31
5091,4689b1816de42507a7d63a4617383c59,7,voucher,1,56.42
11673,4689b1816de42507a7d63a4617383c59,8,voucher,1,14.10
81055,4689b1816de42507a7d63a4617383c59,9,voucher,1,14.10
31004,4689b1816de42507a7d63a4617383c59,10,voucher,1,42.31


In [37]:
# Listagem de mais outro 'order_id' que contém o 'payment_sequential' 14
payments.loc[payments['order_id'] == "ccf804e764ed5650cd8759557269dc13"].sort_values(by='payment_sequential')

,order_id,payment_sequential,payment_type,payment_installments,payment_value
14362,ccf804e764ed5650cd8759557269dc13,1,credit_card,1,2.20
38001,ccf804e764ed5650cd8759557269dc13,2,voucher,1,1.45
40387,ccf804e764ed5650cd8759557269dc13,3,voucher,1,1.15
95153,ccf804e764ed5650cd8759557269dc13,4,voucher,1,0.99
40369,ccf804e764ed5650cd8759557269dc13,5,voucher,1,2.69
21447,ccf804e764ed5650cd8759557269dc13,6,voucher,1,2.26
41414,ccf804e764ed5650cd8759557269dc13,7,voucher,1,1.60
79324,ccf804e764ed5650cd8759557269dc13,8,voucher,1,2.42
87981,ccf804e764ed5650cd8759557269dc13,9,voucher,1,0.97
71309,ccf804e764ed5650cd8759557269dc13,10,voucher,1,1.20


##### Como explicar 'payment_sequential' maior do que 5, se só há 5 formas de pagamento? 
> Veja os dois casos acima:
>> A primeira forma de pagamento é 'credit_card'.

>> Todas as subsequentes são 'voucher'.

>> Mande um WhatsApp pra mim, por favor, quem achar que isso tem sentido.

In [38]:
# Mais uns exemplos: listagem dos 5 primeiros 'order_id' que contêm o 'payment_sequential' 9
payments.loc[payments['payment_sequential'] == 9].head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
7517,67d83bd36ec2c7fb557742fb58837659,9,voucher,1,8.00
14352,fedcd9f7ccdc8cba3a18defedd1a5547,9,voucher,1,5.30
16744,d2e18fcf7ecfd667af457a9e7034c86a,9,voucher,1,20.45
17274,fa65dad1b0e818e3ccc5cb0e39231352,9,voucher,1,1.08
19546,bbc7d177c97f61d86a7486d86ed7a8b2,9,voucher,1,2.92


In [39]:
# Listagem de um 'order_id' que contém o 'payment_sequential' 9
payments.loc[payments['order_id'] == "67d83bd36ec2c7fb557742fb58837659"].sort_values(by='payment_sequential')

,order_id,payment_sequential,payment_type,payment_installments,payment_value
14524,67d83bd36ec2c7fb557742fb58837659,1,voucher,1,8.03
27698,67d83bd36ec2c7fb557742fb58837659,2,voucher,1,8.00
40179,67d83bd36ec2c7fb557742fb58837659,3,voucher,1,8.00
66630,67d83bd36ec2c7fb557742fb58837659,4,voucher,1,8.00
48507,67d83bd36ec2c7fb557742fb58837659,5,voucher,1,8.00
9801,67d83bd36ec2c7fb557742fb58837659,6,voucher,1,8.00
6361,67d83bd36ec2c7fb557742fb58837659,7,voucher,1,1.27
81984,67d83bd36ec2c7fb557742fb58837659,8,voucher,1,8.00
7517,67d83bd36ec2c7fb557742fb58837659,9,voucher,1,8.00
18880,67d83bd36ec2c7fb557742fb58837659,10,voucher,1,0.74


In [40]:
# 1d251ab94983c4adb11e4b168abb1439
# fedcd9f7ccdc8cba3a18defedd1a5547
# 736e1922ae60d0d6a89247b851902527
# Listagem de outro 'order_id' que contém o 'payment_sequential' 9
payments.loc[payments['order_id'] == "fedcd9f7ccdc8cba3a18defedd1a5547"].sort_values(by='payment_sequential')

,order_id,payment_sequential,payment_type,payment_installments,payment_value
9665,fedcd9f7ccdc8cba3a18defedd1a5547,1,credit_card,1,1.67
27087,fedcd9f7ccdc8cba3a18defedd1a5547,2,voucher,1,7.76
87783,fedcd9f7ccdc8cba3a18defedd1a5547,3,voucher,1,26.94
45197,fedcd9f7ccdc8cba3a18defedd1a5547,4,voucher,1,10.33
36869,fedcd9f7ccdc8cba3a18defedd1a5547,5,voucher,1,9.76
103569,fedcd9f7ccdc8cba3a18defedd1a5547,6,voucher,1,8.60
95493,fedcd9f7ccdc8cba3a18defedd1a5547,7,voucher,1,11.78
15930,fedcd9f7ccdc8cba3a18defedd1a5547,8,voucher,1,31.43
14352,fedcd9f7ccdc8cba3a18defedd1a5547,9,voucher,1,5.30
44641,fedcd9f7ccdc8cba3a18defedd1a5547,10,voucher,1,13.27


###### Incrível.
> Mesmo efeito: um monte de 'payment_sequential' de voucher. Não faz sentido!

#### Sobre as repetições de 'payment_type'.  
A explicação que encontro (não recebi WhatsApp algum) é que o cliente pagou com 18 vouchers a compra acima e completou o valor o credit_card.  
E nosso Olist Database optou por criar um registro para cada voucher, ao invés de criar uma coluna com a contagem.  
Talvez precise voltar na tabela <font color=red>"Itens"</font> para rever as deleções que fizemos.  
:(

##### 4.2.2b - Como usar essa configuração de dados

In [41]:
# Qual a chave primária da tabela "payments"?
# Vamos deletar duplicados da concatenação de 'order_id' com 'payment_sequential' (Se não houver deleção, eis a chave primária)
len(payments.drop_duplicates (subset=['order_id', 'payment_sequential']))

103886

###### Pronto: a chave primária da tabela <font color=red>"payment"</font> é a concatenação de 'order_id' e de 'payment_sequential'
> Mas pra que quero um 'payment_sequential' irreal?

###### O ímpeto inicial é deletar a coluna 'payment_sequential'
> O que vou perder?
>> a. Perco o modo de pagamento, que pode ser significativo em uma classificação, que é o objetivo desse trabalho.  
>> b. Também perco o valor do pagamento.


> O que fazer para não perder essas informações?  
>> 1 - Eliminar as redundâncias de voucher. Ou  
>> 2 - Exceto pelo campo 'order_id', transformar as instâncias dos campos em outros campos que indiquem quantidade das instâncias.

> 1 é mais fácil e rápido. Mas 2 é o certo (e mais demorado).

##### <font color=magenta>RESOLUÇÃO</font> sobre como usar essa configuração de dados 
> Criaremos os seguintes campos (para serem associados a cada 'order_id'):
>> 1. Qtde de boleto  
2. Qtde de credit_card  
3. Qtde de debit_card  
4. Qtde de not_defined  
5. Qtde de voucher  
6. Qtde de Installments  
7. Soma de Payment_value

> E desprezaremos os campos 'payment_sequential', 'payment_type', 'payment_installments' e 'payment_value'

In [42]:
# Configuração atual ANTES da implementação da resolução.
Image (url = "cinco tabelas - inclui payments.bmp")

##### 4.2.2c - PREPARAÇÃO da implementação da criação de colunas novas na tabela <font color=red>"payments" </font>
> Criarei uma tabela para cada um dos campos a serem criados.  

In [43]:
payments.shape

(103886, 5)

                                          1. Qtde de 'boleto'

In [44]:
# seleciona 'order_id' cujo 'payment_type' = 'boleto'
payments_somente_boleto = payments.loc[payments['payment_type']=='boleto']

In [45]:
payments_somente_boleto.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
9,0573b5e23cbd798006520e1d5b4c6714,1,boleto,1,51.95
24,8cd68144cdb62dc0d60848cf8616d2a4,1,boleto,1,330.66
29,b69b9260e79a0da00e15f48de1bd2524,1,boleto,1,283.34
39,00d8d65b666158b633f96054d31af43b,1,boleto,1,130.88
42,32b1d8f3789a57f1f752e20ac7404b4c,1,boleto,1,56.60


In [46]:
payments_somente_boleto.shape

(19784, 5)

In [47]:
# Valores únicos de 'order_id'
payments_somente_boleto['order_id'].nunique() # number of unique não inclui os NAN na contagem

19784

###### A 'não repetição' indica haver um - e somente um - boleto para cada 19.784 'order_id'.  

In [48]:
# Mudar o nome da coluna para 'qtde_boleto'
payments_somente_boleto.columns = ['order_id', 'payment_sequential', 'qtde_boleto', 'payment_installments', 'payment_value']

In [49]:
payments_somente_boleto.head()

,order_id,payment_sequential,qtde_boleto,payment_installments,payment_value
9,0573b5e23cbd798006520e1d5b4c6714,1,boleto,1,51.95
24,8cd68144cdb62dc0d60848cf8616d2a4,1,boleto,1,330.66
29,b69b9260e79a0da00e15f48de1bd2524,1,boleto,1,283.34
39,00d8d65b666158b633f96054d31af43b,1,boleto,1,130.88
42,32b1d8f3789a57f1f752e20ac7404b4c,1,boleto,1,56.60


In [50]:
# substituir 'boleto' pelo algarismo '1'.
payments_somente_boleto = payments_somente_boleto.apply(lambda x: x.replace('boleto', 1))

In [51]:
payments_somente_boleto.head()

,order_id,payment_sequential,qtde_boleto,payment_installments,payment_value
9,0573b5e23cbd798006520e1d5b4c6714,1,1,1,51.95
24,8cd68144cdb62dc0d60848cf8616d2a4,1,1,1,330.66
29,b69b9260e79a0da00e15f48de1bd2524,1,1,1,283.34
39,00d8d65b666158b633f96054d31af43b,1,1,1,130.88
42,32b1d8f3789a57f1f752e20ac7404b4c,1,1,1,56.60


In [52]:
payments_somente_boleto.shape

(19784, 5)

In [53]:
# Valores únicos de 'order_id'
payments_somente_boleto['order_id'].nunique() # number of unique não inclui os NAN na contagem

19784

###### DataFrame <font color=red>"payments_somente_boleto"</font> pronto para join

                                          2. Qtde de 'credit_card'

In [54]:
# seleciona 'order_id' cujo 'payment_type' = 'credit_card'
payments_somente_credit_card = payments.loc[payments['payment_type']=='credit_card']

In [55]:
payments_somente_credit_card.shape

(76795, 5)

In [56]:
payments_somente_credit_card.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [57]:
# Valores únicos de 'order_id'
payments_somente_credit_card['order_id'].nunique() # number of unique não inclui os NAN na contagem

76505

###### Esse número (menor do que o de registros da tabela) indica que há alguns 'order_id' que têm mais de uma instância de 'credit_card'.

In [58]:
# Contagem de 'credit_card' de cada 'order_id'
qtde_credit_card = payments_somente_credit_card.groupby('order_id')['payment_type'].count()

df_qtde_credit_card = pd.DataFrame(qtde_credit_card)

df_qtde_credit_card.sort_values(by='payment_type', ascending = False).head()

,payment_type
order_id,
ac40627fa08c6c47cdb3648f3e14901a,2
443038d9ed6b74bc4f7bfa679ce81bf1,2
d16112398f9c83cdef02abee328caadb,2
93185bd1aed7a10299305299fcbebf3d,2
442693e8e705ad1f9c91f762f0d297bc,2


In [59]:
# Tira 'order_id' da condição de índice.
df_qtde_credit_card_index_resetado = df_qtde_credit_card.reset_index()

df_qtde_credit_card_index_resetado.sort_values(by='payment_type', ascending = False).head()

,order_id,payment_type
51364,ac40627fa08c6c47cdb3648f3e14901a,2
20335,443038d9ed6b74bc4f7bfa679ce81bf1,2
62666,d16112398f9c83cdef02abee328caadb,2
43694,93185bd1aed7a10299305299fcbebf3d,2
20323,442693e8e705ad1f9c91f762f0d297bc,2


In [60]:
# Mudar o nome da coluna para 'qtde_credit_card'
df_qtde_credit_card_index_resetado.columns = ['order_id', 'qtde_credit_card']

In [61]:
df_qtde_credit_card_index_resetado.sort_values(by='qtde_credit_card', ascending = False).head()

,order_id,qtde_credit_card
51364,ac40627fa08c6c47cdb3648f3e14901a,2
20335,443038d9ed6b74bc4f7bfa679ce81bf1,2
62666,d16112398f9c83cdef02abee328caadb,2
43694,93185bd1aed7a10299305299fcbebf3d,2
20323,442693e8e705ad1f9c91f762f0d297bc,2


In [62]:
df_qtde_credit_card_index_resetado.shape

(76505, 2)

In [63]:
df_qtde_credit_card_index_resetado['order_id'].nunique() # number of unique não inclui os NAN na contagem

76505

###### DataFrame <font color=red>"df_qtde_credit_card_index_resetado"</font> pronto para join

                                          3. Qtde de 'debit_card'

In [64]:
# seleciona 'order_id' cujo 'payment_type' = 'boleto'
payments_somente_debit_card = payments.loc[payments['payment_type']=='debit_card']

In [65]:
payments_somente_debit_card.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
85,fa2ea4b6e84c1c0fc4f3a6d468c55183,1,debit_card,1,227.12
111,cf61ad2218e5c14397e7c0ba7a6a50f9,1,debit_card,1,35.14
239,76b8568803f110a59ece4167463a4a7e,1,debit_card,1,76.39
251,f553b61e2a7f6dc62c5f8de4ffb4a031,1,debit_card,1,194.91
252,1966886b80f14d0588adf6406afb0026,1,debit_card,1,68.89


In [66]:
payments_somente_debit_card.shape

(1529, 5)

In [67]:
# Valores únicos de 'order_id'
payments_somente_debit_card['order_id'].nunique() # number of unique não inclui os NAN na contagem

1528

###### Há poucos 'order_id'  com mais de uma instância de 'debit_card'. 
> Mas teremos de realizar o groupby para contá-los apropriadamente.

In [68]:
# Contagem de 'debit_card' de cada 'order_id'
qtde_debit_card = payments_somente_debit_card.groupby('order_id')['payment_type'].count()

df_qtde_debit_card = pd.DataFrame(qtde_debit_card)

df_qtde_debit_card.sort_values(by='payment_type', ascending = False).head()

,payment_type
order_id,
a4431cbd79dbddaae7988ce6091cbc3c,2
002f16b7bc4530031b7d90f791b12d8a,1
ad729f060dc2a81423b87ca2845ef6a4,1
ae939729c93b0ecd65e284740cb48997,1
ae6554cd477d704040e7a5ddb5a72886,1


In [69]:
# Tira 'order_id' da condição de índice.
df_qtde_debit_card_index_resetado = df_qtde_debit_card.reset_index()

df_qtde_debit_card_index_resetado.sort_values(by='payment_type', ascending = False).head()

,order_id,payment_type
960,a4431cbd79dbddaae7988ce6091cbc3c,2
0,002f16b7bc4530031b7d90f791b12d8a,1
1015,ad729f060dc2a81423b87ca2845ef6a4,1
1024,ae939729c93b0ecd65e284740cb48997,1
1023,ae6554cd477d704040e7a5ddb5a72886,1


In [70]:
# Mudar o nome da coluna para 'qtde_credit_card'
df_qtde_debit_card_index_resetado.columns = ['order_id', 'qtde_debit_card']

In [71]:
df_qtde_debit_card_index_resetado.sort_values(by='qtde_debit_card', ascending = False).head()

,order_id,qtde_debit_card
960,a4431cbd79dbddaae7988ce6091cbc3c,2
0,002f16b7bc4530031b7d90f791b12d8a,1
1015,ad729f060dc2a81423b87ca2845ef6a4,1
1024,ae939729c93b0ecd65e284740cb48997,1
1023,ae6554cd477d704040e7a5ddb5a72886,1


In [72]:
df_qtde_debit_card_index_resetado.shape

(1528, 2)

In [73]:
df_qtde_debit_card_index_resetado['order_id'].nunique() # number of unique não inclui os NAN na contagem

1528

###### DataFrame <font color=red>"df_qtde_debit_card_index_resetado"</font> pronto para join

                                          4. Qtde de 'not_defined'

In [74]:
# seleciona 'order_id' cujo 'payment_type' = 'not_defined'
payments_somente_not_defined = payments.loc[payments['payment_type']=='not_defined']

In [75]:
payments_somente_not_defined.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
51280,4637ca194b6387e2d538dc89b124b0ee,1,not_defined,1,0.00
57411,00b1cb0320190ca0daa2c88b35206009,1,not_defined,1,0.00
94427,c8c528189310eaa44a745b8d9d26908b,1,not_defined,1,0.00


In [76]:
payments_somente_not_defined.shape

(3, 5)

In [77]:
# Valores únicos de 'order_id'
payments_somente_not_defined['order_id'].nunique() # number of unique não inclui os NAN na contagem

3

In [78]:
# Mudar o nome da coluna para 'qtde_not_defined'
payments_somente_not_defined.columns = ['order_id', 'payment_sequential', 'qtde_not_defined', 'payment_installments', 'payment_value']

In [79]:
# substituir 'not_defined' pelo algarismo '1'.
payments_somente_not_defined = payments_somente_not_defined.apply(lambda x: x.replace('not_defined', 1))

In [80]:
payments_somente_not_defined.head()

,order_id,payment_sequential,qtde_not_defined,payment_installments,payment_value
51280,4637ca194b6387e2d538dc89b124b0ee,1,1,1,0.00
57411,00b1cb0320190ca0daa2c88b35206009,1,1,1,0.00
94427,c8c528189310eaa44a745b8d9d26908b,1,1,1,0.00


###### DataFrame <font color=red>"payments_somente_not_defined"</font> pronto para join

                                          5. Qtde de 'voucher'

In [81]:
# seleciona 'order_id' cujo 'payment_type' = 'voucher'
payments_somente_voucher = payments.loc[payments['payment_type']=='voucher']

In [82]:
payments_somente_voucher.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
25,5cfd514482e22bc992e7693f0e3e8df7,2,voucher,1,45.17
68,947e9c6bfd83a06953c420ef92d3da6e,1,voucher,1,69.46
70,e3e9e3295f4e1512167c232a0aaae8a5,1,voucher,1,50.80
75,3689194c14ad4e2e7361ebd1df0e77b0,2,voucher,1,57.53
102,21b8b46679ea6482cbf911d960490048,2,voucher,1,43.12


In [83]:
payments_somente_voucher.shape

(5775, 5)

In [84]:
# Valores únicos de 'order_id'
payments_somente_voucher['order_id'].nunique() # number of unique não inclui os NAN na contagem

3866

###### Há vários 'order_id'  com mais de uma instância de 'voucher'. 
> E teremos de realizar o groupby para contá-los apropriadamente.

In [85]:
# Contagem de 'voucher' de cada 'order_id'
qtde_voucher = payments_somente_voucher.groupby('order_id')['payment_type'].count()

df_qtde_voucher = pd.DataFrame(qtde_voucher)

df_qtde_voucher.sort_values(by='payment_type', ascending = False).head()

,payment_type
order_id,
fa65dad1b0e818e3ccc5cb0e39231352,29
ccf804e764ed5650cd8759557269dc13,25
285c2e15bebd4ac83635ccc563dc71f4,21
895ab968e7bb0d5659d16cd74cd1650c,21
ee9ca989fc93ba09a6eddc250ce01742,18


In [86]:
# Tira 'order_id' da condição de índice.
df_qtde_voucher_index_resetado = df_qtde_voucher.reset_index()

df_qtde_voucher_index_resetado.sort_values(by='payment_type', ascending = False).head()

,order_id,payment_type
3777,fa65dad1b0e818e3ccc5cb0e39231352,29
3117,ccf804e764ed5650cd8759557269dc13,25
620,285c2e15bebd4ac83635ccc563dc71f4,21
2150,895ab968e7bb0d5659d16cd74cd1650c,21
3616,ee9ca989fc93ba09a6eddc250ce01742,18


In [87]:
# Mudar o nome da coluna para 'qtde_credit_card'
df_qtde_voucher_index_resetado.columns = ['order_id', 'qtde_voucher']

In [88]:
df_qtde_voucher_index_resetado.sort_values(by='qtde_voucher', ascending = False).head()

,order_id,qtde_voucher
3777,fa65dad1b0e818e3ccc5cb0e39231352,29
3117,ccf804e764ed5650cd8759557269dc13,25
620,285c2e15bebd4ac83635ccc563dc71f4,21
2150,895ab968e7bb0d5659d16cd74cd1650c,21
3616,ee9ca989fc93ba09a6eddc250ce01742,18


In [89]:
df_qtde_voucher_index_resetado.shape

(3866, 2)

In [90]:
df_qtde_voucher_index_resetado['order_id'].nunique() # number of unique não inclui os NAN na contagem

3866

###### DataFrame <font color=red>"df_qtde_voucher_index_resetado"</font> pronto para join

                                          6. Soma de 'payment_value'

In [91]:
# Contagem de 'payment_installments' de cada 'order_id'
soma_payment_value = payments.groupby('order_id')['payment_value'].sum()

df_soma_payment_value = pd.DataFrame(soma_payment_value)

df_soma_payment_value.sort_values(by='payment_value', ascending = False).head(45)

,payment_value
order_id,
03caa2c082116e1d31e67e9ae3700499,"13,664.08"
736e1922ae60d0d6a89247b851902527,"7,274.88"
0812eb902a67711a1cb742b3cdaa65ae,"6,929.31"
fefacc66af859508bf1a7934eab1e97f,"6,922.21"
f5136e38d1a14a4dbd87dff67da82701,"6,726.66"
2cc9089445046817a7539d90805e6e5a,"6,081.54"
a96610ab360d42a2e5335a3998b4718a,"4,950.34"
b4c4b76c642808cbe472a32b86cddc95,"4,809.44"
199af31afc78c699f0dbf71fb178d4d4,"4,764.34"


In [92]:
# Tira 'order_id' da condição de índice.
df_soma_payment_value_index_resetado = df_soma_payment_value.reset_index()

df_soma_payment_value_index_resetado.sort_values(by='payment_value', ascending = False).head()

,order_id,payment_value
1471,03caa2c082116e1d31e67e9ae3700499,"13,664.08"
44830,736e1922ae60d0d6a89247b851902527,"7,274.88"
3156,0812eb902a67711a1cb742b3cdaa65ae,"6,929.31"
99071,fefacc66af859508bf1a7934eab1e97f,"6,922.21"
95186,f5136e38d1a14a4dbd87dff67da82701,"6,726.66"


In [93]:
df_soma_payment_value_index_resetado.shape

(99440, 2)

In [94]:
df_soma_payment_value_index_resetado['order_id'].nunique()

99440

In [95]:
# fedcd9f7ccdc8cba3a18defedd1a5547
df_soma_payment_value_index_resetado.loc[df_soma_payment_value_index_resetado['order_id'] == "fedcd9f7ccdc8cba3a18defedd1a5547"].sort_values(by='payment_value')

,order_id,payment_value
99020,fedcd9f7ccdc8cba3a18defedd1a5547,205.74


###### DataFrame <font color=red>"df_soma_payment_value_index_resetado"</font> pronto para join

                                          7. Qtde de 'installments'

In [96]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


###### Aqui complica um pouco. Não sei direito o que fazer:
> <font color=blue>Pergunta 1</font>: qual a qtde de 'installments' do 'order_id' abaixo que tem 21 registros (com iguais valores de 'payment_installments')?
>> Se usar 'count()' ou 'sum()' vou obter o errôneo resultado de 21.

In [97]:
# Listagem de outro 'order_id' especificado
payments.loc[payments['order_id'] == "fedcd9f7ccdc8cba3a18defedd1a5547"].sort_values(by='payment_sequential')

,order_id,payment_sequential,payment_type,payment_installments,payment_value
9665,fedcd9f7ccdc8cba3a18defedd1a5547,1,credit_card,1,1.67
27087,fedcd9f7ccdc8cba3a18defedd1a5547,2,voucher,1,7.76
87783,fedcd9f7ccdc8cba3a18defedd1a5547,3,voucher,1,26.94
45197,fedcd9f7ccdc8cba3a18defedd1a5547,4,voucher,1,10.33
36869,fedcd9f7ccdc8cba3a18defedd1a5547,5,voucher,1,9.76
103569,fedcd9f7ccdc8cba3a18defedd1a5547,6,voucher,1,8.60
95493,fedcd9f7ccdc8cba3a18defedd1a5547,7,voucher,1,11.78
15930,fedcd9f7ccdc8cba3a18defedd1a5547,8,voucher,1,31.43
14352,fedcd9f7ccdc8cba3a18defedd1a5547,9,voucher,1,5.30
44641,fedcd9f7ccdc8cba3a18defedd1a5547,10,voucher,1,13.27


###### Continuando...
> <font color=blue>Pergunta 2</font>: qual a qtde de 'installments' do 'order_id' abaixo que tem 3 registros (com diferentes valores de 'payment_installments')?
>> Se usar 'count()' terei o errôneo resultado 3   
>> se usar 'sum()' terei o errôneo resultado 11

In [98]:
# Listagem de outro 'order_id' especificado
payments.loc[payments['order_id'] == "1d251ab94983c4adb11e4b168abb1439"].sort_values(by='payment_sequential')

,order_id,payment_sequential,payment_type,payment_installments,payment_value
93089,1d251ab94983c4adb11e4b168abb1439,1,credit_card,6,118.56
15259,1d251ab94983c4adb11e4b168abb1439,2,credit_card,4,65.96
32551,1d251ab94983c4adb11e4b168abb1439,3,voucher,1,53.89


###### Bem...

###### Looks like I just got an WhatsApp from myself
> <font color=blue>A resposta a ambas as perguntas é: o número máximo.</font>
>> Na pergunta 1, o número máximo é 1, então o número de prestações é 1, mesmo que haja 21 registros.  
>> Na pergunta 2, o número máximo é 6 (dentre 6, 4 e 1), então consideraremos este como número de prestações.


In [99]:
# Contagem de 'payment_installments' de cada 'order_id'
# Usaremos a função 'max()'
qtde_installments = payments.groupby('order_id')['payment_installments'].max()

df_qtde_installments = pd.DataFrame(qtde_installments)

df_qtde_installments.sort_values(by='payment_installments', ascending = False).head(25)

,payment_installments
order_id,
ffb18bf111fa70edf316eb0390427986,24
fe808fc011ee4ae41f2ed8d1d52b6670,24
70b7e94ea46d3e8b5bc12a50186edaf0,24
859f516f2fc3f95772e63c5757ab0d5b,24
1800f4a4362310d7eb38281634f6dfd1,24
ff36cbc44b8f228e0449c92ef089c843,24
6ae2e8b8fac02522481d2a2f4ca4412c,24
fcbb6af360b31b05460c2c8e524588c0,24
f60ce04ff8060152c83c7c97e246d6a8,24


In [100]:
# 90f864fe19d11549fa01eb81c4dd87e3 tem 2 prestações
# 2b7dbe9be72b8f9733844c31055c0825 tem 2 prestações
# Listagem de 'order_id' especificado
payments.loc[payments['order_id'] == "1800f4a4362310d7eb38281634f6dfd1"].sort_values(by='payment_sequential')

,order_id,payment_sequential,payment_type,payment_installments,payment_value
82431,1800f4a4362310d7eb38281634f6dfd1,1,credit_card,24,325.37


In [101]:
# Tira 'order_id' da condição de índice.
df_qtde_installments_index_resetado = df_qtde_installments.reset_index()

df_qtde_installments_index_resetado.sort_values(by='payment_installments', ascending = False).head()

,order_id,payment_installments
99329,ffb18bf111fa70edf316eb0390427986,24
98884,fe808fc011ee4ae41f2ed8d1d52b6670,24
43794,70b7e94ea46d3e8b5bc12a50186edaf0,24
51756,859f516f2fc3f95772e63c5757ab0d5b,24
9290,1800f4a4362310d7eb38281634f6dfd1,24


In [102]:
df_qtde_installments_index_resetado.shape

(99440, 2)

In [103]:
df_qtde_installments_index_resetado['order_id'].nunique()

99440

###### DataFrame <font color=red>"df_qtde_installments_index_resetado"</font> pronto para join

                                 ------------------ x -------------------

                                 ------------------ x -------------------

##### 4.2.2d - IMPLEMENTAÇÃO da criação de colunas novas na tabela <font color=red>"payments" </font>
> A movimentação dos dados para a tabela <font color=red>"payments" </font> será feita por 'merge'

In [104]:
payments.head(1)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33


In [105]:
# deleção de todos os campos que não sejam 'order_id'
payments_somente_order = payments.drop(['payment_sequential', 'payment_type', 'payment_installments', 'payment_value'], axis=1)
payments_somente_order.head(1)

,order_id
0,b81ef226f3fe1789b1e8b2acac839d17


###### Os DataFrames usados no join são:
> 1. payments_somente_boleto   
2. df_qtde_credit_card_index_resetado  
3. df_qtde_debit_card_index_resetado  
4. payments_somente_not_defined  
5. df_qtde_voucher_index_resetado  
6. df_soma_payment_value_index_resetado  
7. df_qtde_installments_index_resetado

                                     1.Merge para obter 'qtde_boleto'

In [106]:
payments_somente_boleto.head(1)

,order_id,payment_sequential,qtde_boleto,payment_installments,payment_value
9,0573b5e23cbd798006520e1d5b4c6714,1,1,1,51.95


In [107]:
# exemplo de merge  --->    m = pd.merge(tabela_1, tabela_2, how = 'left', on = 'Nome')
payments_boleto = pd.merge(payments_somente_order, payments_somente_boleto, how = 'left', on = 'order_id')

In [108]:
payments_boleto.shape

(103886, 5)

In [109]:
payments_boleto.head()

,order_id,payment_sequential,qtde_boleto,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,nan,nan,nan,nan
1,a9810da82917af2d9aefd1278f1dcfa0,nan,nan,nan,nan
2,25e8ea4e93396b6fa0d3dd708e76c1bd,nan,nan,nan,nan
3,ba78997921bbcdc1373bb41e913ab953,nan,nan,nan,nan
4,42fdf880ba16b47b59251dd489d4441a,nan,nan,nan,nan


In [110]:
# Deleção dos campos não mais necessários
payments_boleto_limpo = payments_boleto.drop(['payment_sequential', 'payment_installments','payment_value'], axis=1)
payments_boleto_limpo.tail()

,order_id,qtde_boleto
103881,0406037ad97740d563a178ecc7a2075c,1.00
103882,7b905861d7c825891d6347454ea7863f,nan
103883,32609bbb3dd69b3c066a6860554a77bf,nan
103884,b8b61059626efa996a60be9bb9320e10,nan
103885,28bbae6599b09d39ca406b747b6632b1,1.00


                                     2.Merge para obter 'qtde_credit_card'

In [111]:
df_qtde_credit_card_index_resetado.head(1)

,order_id,qtde_credit_card
0,00010242fe8c5a6d1ba2dd792cb16214,1


In [112]:
boleto_credit_card = pd.merge(payments_boleto_limpo, df_qtde_credit_card_index_resetado, how = 'left', on = 'order_id')
boleto_credit_card.head()

,order_id,qtde_boleto,qtde_credit_card
0,b81ef226f3fe1789b1e8b2acac839d17,nan,1.00
1,a9810da82917af2d9aefd1278f1dcfa0,nan,1.00
2,25e8ea4e93396b6fa0d3dd708e76c1bd,nan,1.00
3,ba78997921bbcdc1373bb41e913ab953,nan,1.00
4,42fdf880ba16b47b59251dd489d4441a,nan,1.00


In [113]:
boleto_credit_card.shape

(103886, 3)

                                     3.Merge para obter 'qtde_debit_card'

In [114]:
df_qtde_debit_card_index_resetado.head(1)

,order_id,qtde_debit_card
0,002f16b7bc4530031b7d90f791b12d8a,1


In [115]:
boleto_credit_e_debit_card = pd.merge(boleto_credit_card, df_qtde_debit_card_index_resetado, how = 'left', on = 'order_id')
boleto_credit_e_debit_card.head()

,order_id,qtde_boleto,qtde_credit_card,qtde_debit_card
0,b81ef226f3fe1789b1e8b2acac839d17,nan,1.00,nan
1,a9810da82917af2d9aefd1278f1dcfa0,nan,1.00,nan
2,25e8ea4e93396b6fa0d3dd708e76c1bd,nan,1.00,nan
3,ba78997921bbcdc1373bb41e913ab953,nan,1.00,nan
4,42fdf880ba16b47b59251dd489d4441a,nan,1.00,nan


In [116]:
boleto_credit_e_debit_card.shape

(103886, 4)

                                     4.Merge para obter 'qtde_not_defined'

In [117]:
payments_somente_not_defined.head(1)

,order_id,payment_sequential,qtde_not_defined,payment_installments,payment_value
51280,4637ca194b6387e2d538dc89b124b0ee,1,1,1,0.00


In [118]:
boleto_credit_debit_NotDefined = pd.merge(boleto_credit_e_debit_card, payments_somente_not_defined, how = 'left', on = 'order_id')
boleto_credit_debit_NotDefined.head()

,order_id,qtde_boleto,qtde_credit_card,qtde_debit_card,payment_sequential,qtde_not_defined,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,nan,1.00,nan,nan,nan,nan,nan
1,a9810da82917af2d9aefd1278f1dcfa0,nan,1.00,nan,nan,nan,nan,nan
2,25e8ea4e93396b6fa0d3dd708e76c1bd,nan,1.00,nan,nan,nan,nan,nan
3,ba78997921bbcdc1373bb41e913ab953,nan,1.00,nan,nan,nan,nan,nan
4,42fdf880ba16b47b59251dd489d4441a,nan,1.00,nan,nan,nan,nan,nan


In [119]:
# Deleção dos campos não mais necessários
boleto_credit_debit_NotDefined_limpo = boleto_credit_debit_NotDefined.drop(['payment_sequential', 'payment_installments','payment_value'], axis=1)
boleto_credit_debit_NotDefined_limpo.tail()

,order_id,qtde_boleto,qtde_credit_card,qtde_debit_card,qtde_not_defined
103881,0406037ad97740d563a178ecc7a2075c,1.00,nan,nan,nan
103882,7b905861d7c825891d6347454ea7863f,nan,1.00,nan,nan
103883,32609bbb3dd69b3c066a6860554a77bf,nan,1.00,nan,nan
103884,b8b61059626efa996a60be9bb9320e10,nan,1.00,nan,nan
103885,28bbae6599b09d39ca406b747b6632b1,1.00,nan,nan,nan


In [120]:
boleto_credit_debit_NotDefined_limpo.shape

(103886, 5)

                                     5.Merge para obter 'qtde_voucher'

In [121]:
df_qtde_voucher_index_resetado.head(1)

,order_id,qtde_voucher
0,0016dfedd97fc2950e388d2971d718c7,1


In [122]:
bol_cred_deb_Ndef_voucher = pd.merge(boleto_credit_debit_NotDefined_limpo, df_qtde_voucher_index_resetado, how = 'left', on = 'order_id')
bol_cred_deb_Ndef_voucher.head()

,order_id,qtde_boleto,qtde_credit_card,qtde_debit_card,qtde_not_defined,qtde_voucher
0,b81ef226f3fe1789b1e8b2acac839d17,nan,1.00,nan,nan,nan
1,a9810da82917af2d9aefd1278f1dcfa0,nan,1.00,nan,nan,nan
2,25e8ea4e93396b6fa0d3dd708e76c1bd,nan,1.00,nan,nan,nan
3,ba78997921bbcdc1373bb41e913ab953,nan,1.00,nan,nan,nan
4,42fdf880ba16b47b59251dd489d4441a,nan,1.00,nan,nan,nan


In [123]:
bol_cred_deb_Ndef_voucher.shape

(103886, 6)

                                     6.Merge para obter 'soma_payment'

In [124]:
df_soma_payment_value_index_resetado.head(1)

,order_id,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,72.19


In [125]:
bol_cred_deb_Ndef_vou_pay = pd.merge(bol_cred_deb_Ndef_voucher, df_soma_payment_value_index_resetado, how = 'left', on = 'order_id')
bol_cred_deb_Ndef_vou_pay.head()

,order_id,qtde_boleto,qtde_credit_card,qtde_debit_card,qtde_not_defined,qtde_voucher,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,nan,1.00,nan,nan,nan,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,nan,1.00,nan,nan,nan,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,nan,1.00,nan,nan,nan,65.71
3,ba78997921bbcdc1373bb41e913ab953,nan,1.00,nan,nan,nan,107.78
4,42fdf880ba16b47b59251dd489d4441a,nan,1.00,nan,nan,nan,128.45


In [126]:
# renomeando a última coluna:
bol_cred_deb_Ndef_vou_pay.columns = ['order_id', 'qtde_boleto', 'qtde_credit_card', 'qtde_debit_card', 'qtde_not_defined', 'qtde_voucher', 'soma_payment']
bol_cred_deb_Ndef_vou_pay.head()

,order_id,qtde_boleto,qtde_credit_card,qtde_debit_card,qtde_not_defined,qtde_voucher,soma_payment
0,b81ef226f3fe1789b1e8b2acac839d17,nan,1.00,nan,nan,nan,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,nan,1.00,nan,nan,nan,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,nan,1.00,nan,nan,nan,65.71
3,ba78997921bbcdc1373bb41e913ab953,nan,1.00,nan,nan,nan,107.78
4,42fdf880ba16b47b59251dd489d4441a,nan,1.00,nan,nan,nan,128.45


In [127]:
bol_cred_deb_Ndef_vou_pay.shape

(103886, 7)

                                     7.Merge para obter 'qtde_installments'

In [128]:
df_qtde_installments_index_resetado.head(1)

,order_id,payment_installments
0,00010242fe8c5a6d1ba2dd792cb16214,2


In [129]:
bol_cred_deb_Ndef_vou_pay_install = pd.merge(bol_cred_deb_Ndef_vou_pay, df_qtde_installments_index_resetado, how = 'left', on = 'order_id')
bol_cred_deb_Ndef_vou_pay_install.head()

,order_id,qtde_boleto,qtde_credit_card,qtde_debit_card,qtde_not_defined,qtde_voucher,soma_payment,payment_installments
0,b81ef226f3fe1789b1e8b2acac839d17,nan,1.00,nan,nan,nan,99.33,8
1,a9810da82917af2d9aefd1278f1dcfa0,nan,1.00,nan,nan,nan,24.39,1
2,25e8ea4e93396b6fa0d3dd708e76c1bd,nan,1.00,nan,nan,nan,65.71,1
3,ba78997921bbcdc1373bb41e913ab953,nan,1.00,nan,nan,nan,107.78,8
4,42fdf880ba16b47b59251dd489d4441a,nan,1.00,nan,nan,nan,128.45,2


In [130]:
# renomeando a última coluna:
bol_cred_deb_Ndef_vou_pay_install.columns = ['order_id', 'qtde_boleto', 'qtde_credit_card', 'qtde_debit_card', 'qtde_not_defined', 'qtde_voucher', 'soma_payment', 'qtde_installments']
bol_cred_deb_Ndef_vou_pay_install.head()

,order_id,qtde_boleto,qtde_credit_card,qtde_debit_card,qtde_not_defined,qtde_voucher,soma_payment,qtde_installments
0,b81ef226f3fe1789b1e8b2acac839d17,nan,1.00,nan,nan,nan,99.33,8
1,a9810da82917af2d9aefd1278f1dcfa0,nan,1.00,nan,nan,nan,24.39,1
2,25e8ea4e93396b6fa0d3dd708e76c1bd,nan,1.00,nan,nan,nan,65.71,1
3,ba78997921bbcdc1373bb41e913ab953,nan,1.00,nan,nan,nan,107.78,8
4,42fdf880ba16b47b59251dd489d4441a,nan,1.00,nan,nan,nan,128.45,2


In [131]:
bol_cred_deb_Ndef_vou_pay_install.shape

(103886, 8)

                                 ------------------ x -------------------

                                 ------------------ x -------------------

###### Determinação dos valores únicos desses campos

In [132]:
bol_cred_deb_Ndef_vou_pay_install['order_id'].nunique()

99440

In [133]:
bol_cred_deb_Ndef_vou_pay_install['qtde_boleto'].nunique()

1

In [134]:
bol_cred_deb_Ndef_vou_pay_install['qtde_credit_card'].nunique()

2

In [135]:
bol_cred_deb_Ndef_vou_pay_install['qtde_debit_card'].nunique()

2

In [136]:
bol_cred_deb_Ndef_vou_pay_install['qtde_not_defined'].nunique()

1

In [137]:
bol_cred_deb_Ndef_vou_pay_install['qtde_voucher'].nunique()

19

In [138]:
bol_cred_deb_Ndef_vou_pay_install['soma_payment'].nunique()

28029

In [139]:
bol_cred_deb_Ndef_vou_pay_install['qtde_installments'].nunique()

24

#### 4.2.3 - Apresentação da nova tabela <font color=red>"payments"</font>

> Se chamará <font color=red>"payments_normalizada"</font>

> Consistirá da tabela <font color=red>"bol_cred_deb_Ndef_vou_pay_install"</font> (recém obtida), sem duplicidade de
'order_id'

In [140]:
# Deleção dos duplicados de 'order_id' para torná-lo a chave primária da nova tabela
payments_normalizada = bol_cred_deb_Ndef_vou_pay_install.drop_duplicates (subset=['order_id']) # o default é manter a primeira ocorrência da duplicação
payments_normalizada.head()

,order_id,qtde_boleto,qtde_credit_card,qtde_debit_card,qtde_not_defined,qtde_voucher,soma_payment,qtde_installments
0,b81ef226f3fe1789b1e8b2acac839d17,nan,1.00,nan,nan,nan,99.33,8
1,a9810da82917af2d9aefd1278f1dcfa0,nan,1.00,nan,nan,nan,24.39,1
2,25e8ea4e93396b6fa0d3dd708e76c1bd,nan,1.00,nan,nan,nan,65.71,1
3,ba78997921bbcdc1373bb41e913ab953,nan,1.00,nan,nan,nan,107.78,8
4,42fdf880ba16b47b59251dd489d4441a,nan,1.00,nan,nan,nan,128.45,2


In [141]:
payments_normalizada.shape

(99440, 8)

In [142]:
payments_normalizada['order_id'].nunique()

99440

In [143]:
payments_normalizada['qtde_boleto'].nunique()

1

In [144]:
payments_normalizada['qtde_credit_card'].nunique()

2

In [145]:
payments_normalizada['qtde_debit_card'].nunique()

2

In [146]:
payments_normalizada['qtde_not_defined'].nunique()

1

In [147]:
payments_normalizada['qtde_voucher'].nunique()

19

In [148]:
payments_normalizada['soma_payment'].nunique()

28029

In [149]:
payments_normalizada['qtde_installments'].nunique()

24

In [150]:
payments_normalizada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99440 entries, 0 to 103885
Data columns (total 8 columns):
order_id             99440 non-null object
qtde_boleto          19784 non-null float64
qtde_credit_card     76505 non-null float64
qtde_debit_card      1528 non-null float64
qtde_not_defined     3 non-null float64
qtde_voucher         3866 non-null float64
soma_payment         99440 non-null float64
qtde_installments    99440 non-null int64
dtypes: float64(6), int64(1), object(1)
memory usage: 6.8+ MB


In [151]:
# Configuração atual APÓS a implementação da resolução.
Image (url = "cinco tabelas - payments_normalizada.bmp")

                                 ------------------ x -------------------

###### Acabou o tempo. As tabelas restantes serão "limpas" em outro notebook.